In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
import pickle


def get_metrics(model, axis, preprocessed_test=None):
    prob_preds = model.predict_proba(x_train)
    performance = roc_auc_score(y_train, prob_preds[:, axis])
    print "TRAINING: " + str(performance)
    
    prob_preds = model.predict_proba(x_test)
    performance = roc_auc_score(y_test, prob_preds[:, axis])
    print "TEST: " + str(performance)

In [3]:
x = np.load("data/x_normalized.npy")
y = np.load("data/y.npy")

In [4]:
lgb_model = lgb.LGBMClassifier(boosting_type='dart',learning_rate=0.1, num_leaves=500, reg_alpha=0, min_child_samples=5, min_child_weight=1e-05, n_estimators=1000, reg_lambda=100, max_depth=15, dropout=0.7)
lgb_model.fit(x, y, verbose=False)

y_pred = lgb_model.predict_proba(x)[:, 1]
performance = roc_auc_score(y, y_pred)
print performance

0.9844128847809971


In [5]:
xgb_model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.6, gamma=1, learning_rate=0.05, max_delta_step=0,
       max_depth=10, min_child_weight=10, missing=None, n_estimators=200,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)
xgb_model.fit(x, y)

y_pred = xgb_model.predict_proba(x)[:, 1]
performance = roc_auc_score(y, y_pred)
print performance

0.9048556274793762


In [15]:
x_true_test = np.load("data/x_2012_normalized.npy")

xgb_predictions = xgb_model.predict_proba(x_true_test)[:, 1]
lgb_predictions = lgb_model.predict_proba(x_true_test)[:, 1]

new_predictions = np.add(0.5*xgb_predictions, 0.5*lgb_predictions)
new_predictions[new_predictions < 0] = 0.0000001
new_predictions[new_predictions > 1] = 0.9999999

# hekhe

ids = [i for i, _ in enumerate(x_true_test)]
ids = np.array(ids)

df = pd.DataFrame()
df['id'] = ids
df['target'] = new_predictions
df.to_csv("final_submission_2_test.csv", index=False)

print df.head()

   id    target
0   0  0.019080
1   1  0.107205
2   2  0.093598
3   3  0.137514
4   4  0.079241
